# Descubra as melhores notas de matemática do ENEM 2016

Você deverá criar um modelo para prever a nota da prova de matemática de quem participou do ENEM 2016. Para isso, usará Python, Pandas, Sklearn e Regression.


## Tópicos

Neste desafio você aprenderá:

- Python
- Pandas
- Sklearn
- Regression

## Detalhes

O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o [Dicionário dos Microdados do Enem 2016](https://s3-us-west-1.amazonaws.com/acceleration-assets-highway/data-science/dicionario-de-dados.zip).

No arquivo test.csv crie um modelo para prever nota da prova de matemática (coluna `NU_NOTA_MT`) de quem participou do ENEM 2016. 

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: `NU_INSCRICAO` e `NU_NOTA_MT`.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

/home/bruno/codenation/data-science-1/venv/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
train = pd.read_csv('../data/train.csv', index_col='Unnamed: 0')
train.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,...,5.0,A,A,A,A,A,A,A,B,D
2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,...,NaN,A,A,C,A,B,A,A,C,A
3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,...,NaN,A,A,A,A,C,A,A,B,A
4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,...,5.0,C,A,A,A,A,D,A,A,A
5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,...,NaN,A,A,A,A,A,A,A,A,A


In [17]:
train.shape

(13730, 166)

In [12]:
train.columns

Index(['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA',
       'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA',
       'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       ...
       'Q041', 'Q042', 'Q043', 'Q044', 'Q045', 'Q046', 'Q047', 'Q048', 'Q049',
       'Q050'],
      dtype='object', length=166)

In [36]:
# Create dataframe with important infos
def create_info(df):
    df_infos = pd.DataFrame({'column': df.columns,
                             'n_missing': df.isna().sum(),
                             'dtype': df.dtypes,
                             'n_unique': df.nunique()}).reset_index().drop('index', axis=1)
    return df_infos

In [37]:
train_infos = create_info(train)
train_infos.head()

,column,n_missing,dtype,n_unique
0,CO_UF_RESIDENCIA,0,int64,27
1,SG_UF_RESIDENCIA,0,object,27
2,NU_IDADE,0,int64,55
3,TP_SEXO,0,object,2
4,TP_ESTADO_CIVIL,528,float64,4


In [29]:
# Visualize columns that seems to have no significance for the training model
large_nunique = train_infos['n_unique'] > 100
thin_nunique = train_infos['n_unique'] == 1
columns_drop = list(train_infos[large_nunique | thin_nunique]['column'])
keep_columns = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',]
for column in keep_columns:
    try:
        columns_drop.remove(column)
    except:
        pass

In [35]:
try:
    train.drop(columns_drop, axis=1, inplace=True)
except:
    pass
train.head()

,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
1,43,RS,24,M,0.0,1,1,43.0,RS,1,...,5.0,A,A,A,A,A,A,A,B,D
2,23,CE,17,F,0.0,3,1,23.0,CE,2,...,NaN,A,A,C,A,B,A,A,C,A
3,23,CE,21,F,0.0,3,1,23.0,CE,3,...,NaN,A,A,A,A,C,A,A,B,A
4,33,RJ,25,F,0.0,0,1,33.0,RJ,1,...,5.0,C,A,A,A,A,D,A,A,A
5,13,AM,28,M,0.0,2,1,15.0,PA,1,...,NaN,A,A,A,A,A,A,A,A,A


In [ ]:
train_infos = create_infos(train)
col_int = train_infos['dtype'] == 'int64'
col_float = train_infos['dtype'] == 'float64'
num_features = list(train_infos[col_int | col_float]['column'])
cat_features